In [1]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from skimage.feature import graycomatrix,hog
scaler = StandardScaler()

import xgboost as xgb
from sklearn.metrics import accuracy_score
import cv2
import glob
from tqdm import tqdm
import time
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # For evaluation metrics
import joblib

In [2]:
pneumonia_image_dir = 'Pneumonia'
non_pneumonia_image_dir = 'Normal'

pneumonia_image_paths = glob.glob(pneumonia_image_dir + '/*.jpeg')  # Adjust file extension if needed
non_pneumonia_image_paths = glob.glob(non_pneumonia_image_dir + '/*.jpeg')

all_image_paths = pneumonia_image_paths + non_pneumonia_image_paths
labels = [1] * len(pneumonia_image_paths) + [0] * len(non_pneumonia_image_paths)

accuracies = []
# Perform K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=None)
for train_index, val_index in kf.split(all_image_paths):
    X_train_paths, X_val_paths = [all_image_paths[i] for i in train_index], [all_image_paths[i] for i in val_index]
    y_train, y_val = [labels[i] for i in train_index], [labels[i] for i in val_index]
    
    # Load and preprocess training images
    X_train = []
    for path in tqdm(X_train_paths, desc="Training Images"):
        image = cv2.imread(path)  # Load image using OpenCV or any other library
        resized_image = cv2.resize(image, (224, 224))
        # (224, 224) is a common size for pre-trained models and feature extraction.
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        # Extract HOG features
        hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
        # Normalize the histogram
        hist /= hist.sum()
        # Compute GLCM-like features
        mean = (hist * np.arange(256)).sum()
        variance = (hist * ((np.arange(256) - mean) ** 2)).sum()
        homogeneity = (hist / (1 + np.abs(np.arange(256) - np.arange(256)[:, None]))).sum()
        contrast = (hist * (np.abs(np.arange(256) - np.arange(256)[:, None]) ** 2)).sum()
        X_train.append([mean, variance, homogeneity, contrast])
        # Append extracted features to X_train or X_val based on the loop iteration
        # normalization
    X_train = scaler.fit_transform(X_train)
    joblib.dump(scaler, 'scaler.pkl')
        # Apply preprocessing steps (resizing, normalization, etc.) to the image
        # Append preprocessed image to X_train
        
        # Load and preprocess validation images
    X_val = []
    for path in tqdm(X_val_paths, desc="Validation Images"):
        image = cv2.imread(path) 
        resized_image = cv2.resize(image, (224, 224))
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        # Extract HOG features
        #fd = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])

    # Normalize the histogram
        hist /= hist.sum()

    # Compute GLCM-like features
        mean = (hist * np.arange(256)).sum()
        variance = (hist * ((np.arange(256) - mean) ** 2)).sum()
        homogeneity = (hist / (1 + np.abs(np.arange(256) - np.arange(256)[:, None]))).sum()
        contrast = (hist * (np.abs(np.arange(256) - np.arange(256)[:, None]) ** 2)).sum()
        X_val.append([mean, variance, homogeneity, contrast])
        # Append extracted features to X_train or X_val based on the loop iteration
        # normalization
    X_val = scaler.transform(X_val)
    
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on validation data
    y_pred = model.predict(X_val)
    
    # Evaluate the model (using multiple metrics)
    accuracy = accuracy_score(y_val, y_pred)
    
    # Print results for this fold
    print(f"Accuracy for this fold: {accuracy}")
    
    # Store results in lists for later analysis
    accuracies.append(accuracy)

# After the loop, assess overall performance    
print("Mean accuracy:", np.mean(accuracies))
    
        

Validation Images: 100%|██████████| 1172/1172 [00:39<00:00, 29.65it/s]


Accuracy for this fold: 0.7261092150170648


Validation Images: 100%|██████████| 1171/1171 [00:16<00:00, 72.44it/s]


Accuracy for this fold: 0.7463706233988044


Validation Images: 100%|██████████| 1171/1171 [00:18<00:00, 64.73it/s]


Accuracy for this fold: 0.71733561058924


Validation Images: 100%|██████████| 1171/1171 [00:16<00:00, 71.42it/s]


Accuracy for this fold: 0.7113578138343296


Validation Images: 100%|██████████| 1171/1171 [00:16<00:00, 70.83it/s]


Accuracy for this fold: 0.7198975234842016
Mean accuracy: 0.724214157264728


In [3]:
model.save_model('xgboost_model.json')

In [4]:
#For testing purpose only
# Assuming model and scaler are already trained and defined

# Load and preprocess a single X-ray image
def preprocess_single_image(image_path, scaler):
    # Load image
    image = cv2.imread(image_path)
    
    # Preprocess image (resize, convert to grayscale, extract HOG features, etc.)
    resized_image = cv2.resize(image, (224, 224))
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Compute GLCM-like features
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    hist /= hist.sum()
    mean = (hist * np.arange(256)).sum()
    variance = (hist * ((np.arange(256) - mean) ** 2)).sum()
    homogeneity = (hist / (1 + np.abs(np.arange(256) - np.arange(256)[:, None]))).sum()
    contrast = (hist * (np.abs(np.arange(256) - np.arange(256)[:, None]) ** 2)).sum()

    # Create feature vector
    X_processed = np.array([mean, variance, homogeneity, contrast]).reshape(1, -1)
    # Ensure the features are in the same format as used during training
    X_processed = scaler.transform(X_processed)  # Assuming you have a trained scaler
    
    return X_processed

# Path to the single X-ray image you want to test
image_path_to_test = 'unseen_P.jpeg'

# Preprocess the single image
X_single_image = preprocess_single_image(image_path_to_test, scaler)

# Make prediction
y_pred_single_image = model.predict(X_single_image)

# If you want probability scores for binary classification
y_proba_single_image = model.predict_proba(X_single_image)[:, 1]

# Display the prediction
print(f"Image: {image_path_to_test}, Prediction: {y_pred_single_image[0]}, Probability: {y_proba_single_image[0]}")


Image: unseen_P.jpeg, Prediction: 1, Probability: 0.9920250177383423
